> When do you use BFS vs DFS? (What is the important property of BFS?)

DFS is implicitly recursive, which enables more elegant solutions for a wide range of problems.

BFS is implicitly iterative. There's no call stack, so it has better memory consumption behavior. It's much easier to debug than a recursive algorithm is.

> https://leetcode.com/problems/number-of-islands/

In [40]:
function fill(grid, pos, idxset) {
    let v = grid[pos[0]][pos[1]];
    let todo = [pos];
    let height = grid.length;
    let width = grid[0].length;
    
    while (todo.length > 0) {
        let next_todo = [];
        for (let pos of todo) {
            let [x, y] = [pos[0], pos[1]];
            for (let new_pos of [[x - 1, y], [x + 1, y], [x, y - 1], [x, y + 1]]) {
                if (new_pos[0] < 0 || new_pos[1] < 0 || new_pos[0] >= height || new_pos[1] >= width) {
                    continue;
                } else if (grid[new_pos[0]][new_pos[1]] !== grid[pos[0]][pos[1]]) {
                    continue;
                } else if (!idxset.has(JSON.stringify(new_pos))) {
                    idxset.add(JSON.stringify(new_pos));
                    next_todo.push(new_pos);
                }
            }
        }
        todo = next_todo;
    }
    
    return idxset;
}


function numIslands(grid) {
    if (grid.length === 0 || grid[0].length === 0) return 0;
    
    let idxset = new Set();
    let n_islands = 0;
    
    for (let x of [...Array(grid.length).keys()]) {
        for (let y of [...Array(grid[0].length).keys()]) {
            let pos = [x, y];
            if (!idxset.has(JSON.stringify(pos))) {
                idxset = fill(grid, pos, idxset);
                n_islands += parseInt(grid[x][y]);
            }
        }
    }
    
    return n_islands;
}

In [31]:
numIslands([["1","1","1","1","0"],["1","1","0","1","0"],["1","1","0","0","0"],["0","0","0","0","0"]])

1

In [41]:
numIslands([[1]])

1

In [43]:
numIslands([[]])

0

* The outer loop visits every index in the array one. The inner loop (cumulatively) visits every single index in the array one. Therefore this algorithm is $O(n^2 + n^2) \approx O(n^2)$. It also requires $O(n^2)$ space, which is used to store the set of indices already visited.
* The theoretical maximum efficiency of a solution to this problem is $O(n^2)$; no matter what we will have to visit every single array location. Therefore we can make only constant factor optimizations to this algorithm.
* The inner loop is BFS. It's possible to rewrite to use DFS.
* One constant time optimization would be to maintain a second set of elements that we subtract from as we see new elements, and instead of iterating through the position array linearly, we iterate through this position set until we exhaust the options.
* Some JavaScript weirdness involved. Empty list, `[]`, evaluates to `true` in JS. And different lists with the same values compare to false, e.g. `[1,2] !== [1,2]`. This requires a length check and the use of `JSON.stringify`, respectively.
* A [more elegant Python solution](https://leetcode.com/problems/number-of-islands/discuss/56340/Python-Simple-DFS-Solution) uses a recursive form of DFS to make that piece of code more elegant, and it omits the need for a indices-visited map by filling in visited values with a sentinel character (`#`). This reduces the memory requirements to $O(1)$&mdash;no further arrays of non-constant length are allocated.

> https://leetcode.com/problems/redundant-connection/

In [172]:
function treeify(edgelist) {
    if (edgelist.length === 0) return null;
    
    class Node { constructor(v) { this.v = v; this.neighbors = []; } }
    
    let nodemap = new Map();
    for (let edge of edgelist) {
        for (let v of edge) {
            if (nodemap.has(v)) {
                n = nodemap.get(v);
            } else {
                nodemap.set(v, new Node(v));
            }
        }
        nodemap.get(edge[0]).neighbors.push(nodemap.get(edge[1]));
        nodemap.get(edge[1]).neighbors.push(nodemap.get(edge[0]));
    }
    return nodemap.get(edgelist[0][0]);
}

function findRedundantConnection(edges) {
    let root = treeify(edges);
    let re = new Set(_traverse(root, []).map(v => JSON.stringify(v)));
    let i = edges.length - 1;
    while (true) {
        let e = edges[i];
        if (re.has(JSON.stringify(e))) {
            return e;
        }
        i -= 1;
    }
}

function _traverse(node, prior_nodes_visited) {
    let cyclic_edges = [];
    
    for (let nn of node.neighbors) {
        // the connections we *just* followed does not count
        if (prior_nodes_visited.length > 0 &&
            prior_nodes_visited[prior_nodes_visited.length - 1].v === nn.v) {
            continue;
        }
        // if one of the other nodes connects, we have a cycle
        if (prior_nodes_visited.some(pn => nn === pn)) {
            // reconstruct the cyclic edges
            // we're doing something wrong here, but what?
            let cyclic_edges = [];
            let [u, v] = [null, null];
            for (let j of [...Array(prior_nodes_visited.length - 1).keys()]) {
                let k = j + 1;
                [u, v] = [prior_nodes_visited[j].v, prior_nodes_visited[k].v]
                u < v ? cyclic_edges.push([u, v]) : cyclic_edge.push([v, u]);
            }
            [u, v] = [prior_nodes_visited[prior_nodes_visited.length - 1].v, node.v];
            u < v ? cyclic_edges.push([u, v]) : cyclic_edge.push([v, u]);
            return cyclic_edges;
        } else {
            return _traverse(nn, [...prior_nodes_visited, node]);
        }
    }
}

In [173]:
findRedundantConnection([[1,2], [1,3], [2,3]])

[ 2, 3 ]

In [174]:
findRedundantConnection([[1,2], [2,3], [3,4], [1,4], [1,5]])

[ 3, 4 ]

* This is the solution I came up with.
* The general idea of this solution is correct. First, the nodes are converted into a tree (because working with edgelists is clunky?). Then we perform DFS on the nodes, stopping as soon as we detect a cycle. A list of prior nodes visited is passed down the function; the values contained therein are used to determine whether or not the current edge is part of a cycle. It then iterates through the edge list backwards to determine which connection is the last cyclic one which may be broken. This is then compared one at a time to the edge list entries in a backwards iteration order to find the last matching edge, and that edge is returned.
* Something is wrong with the implementation, but trying to fix it is just going to burn more time on a problem I have spent a f-ton of time on already, so here we leave it.
* This algorithm is $O(n)$. It visits (up to) every node in the graph in iteration order ($n$ ops); once it discovers a cycle it visits (up to) every node in the graph again to reconstruct the cycle edge list ($n$ ops); then it compares that against the defining edgelist ($n$ ops again). $O(n + n + n) \approx O(n)$.
* This solution methodology is technically correct, in that it can be made to work, but I don't know how to fix this code to make that happen.
* I feel like I completely whiffed on this problem. :(
* The optimal solution is from [this forum post](https://leetcode.com/problems/redundant-connection/discuss/123819/Union-Find-with-Explanations-(Java-Python)):

In [4]:
def findRedundantConnection(edges):
    import pdb; pdb.set_trace()
    
    parent = [0] * len(edges)

    def find(x):
      if parent[x] == 0:
        return x
      parent[x] = find(parent[x])
      return parent[x]

    def union(x, y):
      rootX = find(x)
      rootY = find(y)
      if rootX == rootY:
        return False
      parent[rootX] = rootY
      return True

    for x, y in edges:
      if not union(x - 1, y - 1): 
        return [x, y]

In [7]:
# findRedundantConnection([[1,2], [2,3], [3,4], [1,4], [1,5]])

* This solution is known as a **union find**. It uses the property that in a non-directional graph with a cycle, at least two nodes necessarily have a topmost parent node in common. The non-directionality here is important: a non-directional graph has no identifiable root node, and therefore we may choose to establish the child-parent relationship arbitrarily.
* This is a completely different technique template from DFS. Union find is not in the algo techniques "starter kit", so I'll skip over its use here for now.

In [56]:
function findRedundantConnection(edges) {
    let adj = new Map();    
    for (let edge of edges) {
        adj.has(edge[0]) ?
            adj.set(edge[0], [...adj.get(edge[0]), edge[1]]) :
            adj.set(edge[0], [edge[1]]);
        adj.has(edge[1]) ? 
            adj.set(edge[1], [...adj.get(edge[1]), edge[0]]) : 
            adj.set(edge[1], [edge[0]]);
    }
    let sp = edges[0][0];
    let opts = traverse(adj, sp, [], []);
    opts = opts.map(v => JSON.stringify(v));
    
    let i = edges.length - 1;
    while (true) {
        if (opts.includes(JSON.stringify(edges[i]))) {
            return edges[i];
        } else {
            i -= 1;
        }
    }
}

function traverse(adj, sp, prior_nodes_visited, solutions) {
    let nexts = adj.get(sp);
    let prev = prior_nodes_visited[prior_nodes_visited.length - 1];
    let prior_nodes_visited_set = new Set(prior_nodes_visited);
    for (let next of nexts) {
        if (next === prev) {
            // skip the immediate predecessor node
            continue;
        } else if (prior_nodes_visited_set.has(next)) {
            // cycle detected
            let i = 0;
            while (i < prior_nodes_visited.length - 1) {
                let [u, v] = [prior_nodes_visited[i], prior_nodes_visited[i + 1]];
                if (u > v) [u, v] = [v, u];
                solutions.push([u, v]);
                i += 1;
            }
            let [u, v] = [prev, sp];
            if (u > v) [u, v] = [v, u];
            solutions.push([u, v]);
        } else {
            solutions = traverse(adj, next, [...prior_nodes_visited, sp], solutions);
        }
    }
    return solutions;
}

In [58]:
findRedundantConnection([[1,2],[2,3],[3,4],[1,4],[1,5]])

[ 1, 4 ]

* Better! Some JavaScript annoyances around array membership.

> https://leetcode.com/problems/network-delay-time/ (use djikstra)

* Djikstra's algorithm is a distance-finding algorithm that can be used to find the minimum distance between two nodes, or alternatively the lattice of all distances to a node. I've relearned this algorithm probably three different times at this point...
* We'll start with a BFS solution first.
* ...actually after doing some work, I'm going to suspend this effort. I am starting to understand that I do not understand DFS and BFS on a sufficiently intrinsic level to be able to do this problem in the reasonable time period expected in an interview setting (30 minutes, per my notes). Per our notes on technical interviews, that means I need to go back and spend a day doing Easy problems in this technical class, before returning to and trying for Medium again.
* Also...not having access to tuples in JavaScript is really making me unhappy, and complicating code that should be simple. I should take a minute to relearn symbols, which I'm pretty sure can do for array insertion and access in JavaScript what tuples do for it in JavaScript.
* Since this will be time-consuming, let's suspend our efforts here and skip ahead to.